# Fetch Data

Note book where data is fetched and saved as a .csv to be used for all publication analyses.

Stopped here- do imports, investigate the data and save out. Consider if wanting to use a spoke only, cpoke growth only or probe only df as well.


# 0. Imports


In [60]:
import config as c
import utils as u
import behav_viz.ingest as ingest
import pandas as pd
import glob

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Trial DataFrame

Each row of the df frame is a unique trial for a given session, animal. Note that for V2 animals in the growth phase, a trial collapses failed settling in pokes into a summary (e.g. `avg_failed_cpoke_dur`) and represents the final successful trial in a potential bout of pokes. For a fully unraveled DataFrame, see the `Poke DataFrame`


In [ ]:
# load data for a given data range
tdf = ingest.create_trials_df.create_trials_df_from_dj(
    c.ANIMALS, c.DATE_MIN, c.DATE_MAX
)

# clean up dates
tdf = ingest.drop_dates(tdf, c.DATE_DROPS)  # see config.py for rationale

# remove post probe stages
tdf = tdf.query("stage < 11")

# remove columns not relevant to the analysis
tdf = tdf.drop(columns=c.DROP_COLUMNS)

# save data
tdf.to_parquet("trials_df.parquet", index=False)
tdf.head()

In [5]:
t = u.load_trials_df()

Loading trials data from directory: /Users/jessbreda/Desktop/github/jbreda_animal_training/notebooks/fixation_publication


In [17]:
t.columns

Index(['trial', 'animal_id', 'date', 'sess_id', 'protocol', 'result', 'hits',
       'violations', 'temperror', 'hit_rate', 'error_rate', 'violation_rate',
       'temp_error_rate', 'n_lpokes', 'n_cpokes', 'n_rpokes', 'trial_dur',
       'trial_start', 'first_spoke', 'first_lpoke', 'first_rpoke',
       'n_settling_ins', 'avg_settling_in', 'cpoke_dur', 'stage', 'stage_name',
       'curriculum', 'fix_experiment', 'sides', 'SMA_set',
       'auto_spoke_to_cpoke', 'settling_in_dur', 'legal_cbreak_dur',
       'delay_dur', 'pre_go_dur', 'inter_trial_dur', 'fixation_dur',
       'initial_fixation_dur', 'target_fixation_dur', 'fixation_adjustment',
       'has_reached_target_fixation', 'settling_in_determines_fixation',
       'delay_growth', 'delay_warm_up_on', 'settling_in_growth',
       'settling_in_warm_up_on', 'warm_up_imp', 'go_type', 'go_dur',
       'give_type_set', 'give_type_imp', 'give_frac', 'timeouts',
       'water_delivered', 'give_water_not_drunk', 'crash_hist',
       'was

# 2. Days DataFrame

Each row of the df frame is data from a unique day for a given animal and focus on measures like mass, rig used, total water drunk etc.


In [37]:
ddf = ingest.create_days_df.create_days_df_from_dj(
    animal_ids=c.ANIMALS,
    date_min=c.DATE_MIN,
    date_max=c.DATE_MAX,
    verbose=True,
)

ddf = ingest.drop_dates(ddf, c.DATE_DROPS)

Fetched data from SessionAggDate table from 2024-07-20 to 2024-10-17  
Today is not being queried or data already exists in the database.
No new data from today, 2025-03-03 to add to SessionAggDate.

1365 daily summaries fetched for animals: 
['R040', 'R041', 'R042', 'R043', 'R044', 'R045', 'R046', 'R047', 'R048', 'R049', 'R050', 'R051', 'R052', 'R053', 'R054', 'R055', 'R056', 'R057']
between 2024-07-20 and 2024-10-17
dropping R042 2024-07-29 length: 1365
length: 1364
dropping R043 2024-07-26 length: 1364
length: 1363
dropping R043 2024-07-27 length: 1363
length: 1362
dropping R043 2024-07-28 length: 1362
length: 1361
dropping R043 2024-07-29 length: 1361
length: 1360
dropping R046 2024-08-12 length: 1360
length: 1359
dropping R052 2024-08-14 length: 1359
length: 1358
dropping R051 2024-08-20 length: 1358
length: 1357
dropping R051 2024-08-21 length: 1357
length: 1356
dropping R051 2024-08-22 length: 1356
length: 1355


In [49]:
# Add in stage and fix_experiment information from the trial df & remove post-prone trials (which get by left joining to tdf)

tdf_agg = (
    tdf.groupby(["date", "animal_id"])
    .agg({"stage": "first", "fix_experiment": "first"})
    .reset_index()
)
tdf_agg["date"] = pd.to_datetime(tdf_agg["date"])

ddf_merged = pd.merge(tdf_agg, ddf, on=["date", "animal_id"], how="left")

# save data
ddf_merged.to_parquet("days_df.parquet", index=False)
ddf_merged.head()

,date,animal_id,stage,fix_experiment,session_agg_date_id,num_sessions,n_done_trials,rigid,starttime,endtime,...,num_water,rig_volume,num_rigwater,train_dur_hrs,trial_rate,starttime_hrs,endtime_hrs,side_bias,volume_target,water_diff
0,2024-07-20,R040,1.0,V1,3447340,1,18,Rig01,0 days 15:44:27,0 days 16:01:21,...,1,0.744,1,0.281667,63.91,15.740833,16.022500,-0.99,98.6,-56.528
1,2024-07-20,R041,1.0,V2,3447341,1,147,Rig04,0 days 13:49:28,0 days 16:01:23,...,1,2.670,1,2.198611,66.86,13.824444,16.023056,-0.97,96.2,-93.530
2,2024-07-20,R042,1.0,V1,3447342,1,163,Rig05,0 days 13:49:29,0 days 16:01:18,...,1,8.400,1,2.196944,74.19,13.824722,16.021667,-0.99,105.8,-97.400
3,2024-07-20,R043,1.0,V2,3447343,1,148,Rig06,0 days 13:49:28,0 days 16:01:20,...,1,2.874,1,2.197778,67.34,13.824444,16.022222,-0.89,97.0,-94.126
4,2024-07-20,R044,1.0,V1,3447344,1,144,Rig09,0 days 13:49:33,0 days 16:01:21,...,1,1.896,1,2.196667,65.55,13.825833,16.022500,-0.96,80.0,-78.104


# 3. Poke DataFrame


In [113]:
# Define the folder path
fpath = "/Volumes/brody/jbreda/FixationGrower_cpoke_table/"

# Use glob to find all CSV files in the folder
csv_files = glob.glob(fpath + "*.csv")

# Read each CSV file into a DataFrame and store them in a list
df_list = [pd.read_csv(file) for file in csv_files]

# Concatenate all DataFrames into a single DataFrame
pdf = pd.concat(df_list, ignore_index=True)

# Display the first few rows of the combined DataFrame
pdf.head()


,sessid,animal_id,trial,cpoke_dur,cpoke_iti,post_settling_violation,settling_violation,was_rewarded,fixation_dur,date,stage
0,964851,R043,1,0.3827,NaN,0,1,0,0.401,30-Jul-2024 14:47:25,5.0
1,964851,R043,1,0.1890,0.5006,0,1,0,0.401,30-Jul-2024 14:47:25,5.0
2,964851,R043,1,0.3638,0.8218,0,1,0,0.401,30-Jul-2024 14:47:25,5.0
3,964851,R043,1,0.3416,0.2565,0,1,0,0.401,30-Jul-2024 14:47:25,5.0
4,964851,R043,1,0.0580,0.2194,0,1,0,0.401,30-Jul-2024 14:47:25,5.0


## 3.1 Cleaning


In [114]:
def assign_fix_experiment(animal_id):
    if animal_id in c.V1_ANIMALS:
        return "V1"
    elif animal_id in c.V2_ANIMALS:
        return "V2"
    return None


# Apply mapping & make additional columns
pdf["fix_experiment"] = pdf["animal_id"].map(assign_fix_experiment)


We need to adjust the cpoke iti (the time between c-out to c-in from one trial to the next) to take into account the iti that happens and the end of the trial as well as any additional timeout penalties. Therefore, this adjustment only applies to the **first** cpoke in a trial since it occurs after the last trial has cleaned up and the SMA went through terminal states including penalities and ITI.


In [115]:
# ----------------------------
# 1.  Load trials table
# ----------------------------

tdf = u.load_trials_df()

# ----------------------------
# 2.  On the trials table – create prev_sma_iti  **and** prev_timeout
# ----------------------------
tdf = (
    tdf
    .sort_values(["animal_id", "sess_id", "trial"])
)

tdf["prev_sma_iti"] = (
    tdf
    .groupby(["animal_id", "sess_id"])["inter_trial_dur"]
    .shift(1)
)

tdf["prev_timeout"] = (                           # ← NEW
    tdf
    .groupby(["animal_id", "sess_id"])["timeouts"]
    .shift(1)
    .fillna(0)                                        # treat “no timeout” as 0 s
)

tdf_keep = tdf[[
    "animal_id", "sess_id", "trial",
    "prev_sma_iti", "prev_timeout"                    # ← include new column
]]

# ----------------------------
# 3.  On the cpoke table – flag first cpoke of each trial
# ----------------------------
pdf = (
    pdf
    .sort_values(["animal_id", "sessid", "trial"])
)

pdf["is_first_cpoke"] = (
    pdf
    .groupby(["animal_id", "sessid", "trial"])
    .cumcount()
    == 0
)

# ----------------------------
# 4.  Merge prev_* columns onto the cpoke rows
# ----------------------------
merged_pdf = pdf.merge(
    tdf_keep,
    left_on  = ["animal_id", "sessid", "trial"],
    right_on = ["animal_id", "sess_id", "trial"],
    how = "left"
).drop(columns="sess_id")

# ----------------------------
# 5.  Keep penalties only for the first cpoke of the trial
# ----------------------------
mask_not_first = ~merged_pdf["is_first_cpoke"]
merged_pdf.loc[mask_not_first, ["prev_sma_iti", "prev_timeout"]] = 0

# ----------------------------
# 6.  Compute the adjusted ITI
# ----------------------------
merged_pdf["cpoke_iti_adjusted"] = (
    merged_pdf["cpoke_iti"]
    - merged_pdf["prev_sma_iti"]
    - merged_pdf["prev_timeout"]       # ← subtract extra penalty
)

# # defensive post-processing: clip negatives to NaN
# neg_or_large = (merged_pdf["cpoke_iti_adjusted"] < 0) | (merged_pdf["cpoke_iti_adjusted"] > 7200)
# print(f"Number of negative or large values: {neg_or_large.sum()}")
# merged_pdf.loc[neg_or_large, ["prev_sma_iti", "prev_timeout", "cpoke_iti_adjusted"]] = pd.NA

neg = merged_pdf["cpoke_iti_adjusted"] < 0
print(f"Number of negative values: {neg.sum()}")
merged_pdf.loc[neg, ["prev_sma_iti", "prev_timeout", "cpoke_iti_adjusted"]] = pd.NA

neg = merged_pdf["cpoke_iti_adjusted"] > 7200
print(f"Number of large values: {neg.sum()}")
merged_pdf.loc[neg, ["prev_sma_iti", "prev_timeout", "cpoke_iti_adjusted"]] = pd.NA


# ----------------------------
# 7.  Inspect
# ----------------------------
merged_pdf[[
    "animal_id","sessid","trial",
    "cpoke_iti","prev_sma_iti","prev_timeout","cpoke_iti_adjusted",
    "is_first_cpoke"
]].head()

Loading days data from directory: /Users/jessbreda/Desktop/github/jbreda_animal_training/notebooks/fixation_publication
Number of negative values: 13
Number of large values: 5


,animal_id,sessid,trial,cpoke_iti,prev_sma_iti,prev_timeout,cpoke_iti_adjusted,is_first_cpoke
0,R040,965069,1,NaN,NaN,0.0,NaN,True
1,R040,965069,2,72.409421,12.332821,0.0,60.076600,True
2,R040,965069,3,28.839480,5.777362,0.0,23.062118,True
3,R040,965069,4,12.954300,4.993989,0.0,7.960311,True
4,R040,965069,5,15.057600,4.821803,0.0,10.235797,True


The code below processes behavioral data differently based on the experimental stage to accurately capture trial cpoke iti.

In growing stages, we care about settling-in violations and post-settling in violations because of the differences in curriculum. V2 animals can only violate during the settling in period period, whereas V1 animals can only violate during the post-settling in period.

In probe stages, the settling-in period is fixed at 200 ms and no longer behaviorally relevant for inter-trial interval (ITI) analysis; instead, we focus only on post-settling violations and retain only the final center poke per trial. The processed data from both stages is then combined to enable consistent downstream analysis of valid trials.


In [120]:

g_pdf = merged_pdf.query("stage in @c.GROWING_STAGES").copy()
g_pdf["was_valid"] = (g_pdf["post_settling_violation"] == 0) & (
    g_pdf["settling_violation"] == 0
)
g_pdf["prev_was_valid"] = (
    g_pdf
    .groupby(["animal_id", "date"])["was_valid"]
    .shift(1)
)

penalty_on_stages = [8,9,10]
pr_pdf = merged_pdf.query("stage in @penalty_on_stages").copy()
iti_summed = pr_pdf.groupby(["sessid", "trial"])["cpoke_iti_adjusted"].sum().reset_index().rename(columns={"cpoke_iti_adjusted": "cpoke_iti_adjusted_summed"})

last_pdf = pr_pdf.groupby(["sessid", "trial"]).tail(1)
pr_pdf_no_settling = last_pdf.merge(iti_summed, on=["sessid", "trial"], how="left")

pr_pdf_no_settling["was_valid"] = pr_pdf_no_settling["post_settling_violation"] == 0
pr_pdf_no_settling["prev_was_valid"] = (
    pr_pdf_no_settling
    .groupby(["animal_id", "date"])["was_valid"]
    .shift(1)
)

pdf_cleaned = pd.concat([g_pdf, pr_pdf_no_settling], ignore_index=True)

In [121]:
pdf_cleaned["relative_cpoke_dur"] = pdf_cleaned["cpoke_dur"] - pdf_cleaned["fixation_dur"]
pdf_cleaned.to_parquet("poke_df.parquet", index=False)

# 4. V2 Continued Data


In [13]:
c_tdf = ingest.create_trials_df.create_trials_df_from_dj(
    animal_ids=c.V2_ANIMALS,
    date_min=c.DATE_MIN,
    date_max="2025-03-15",
    verbose=True,
)

[2025-05-10 19:05:44,131][WARNING]: MySQL server has gone away. Reconnecting to the server.


animal_id R041
dropping 2 sessions of 217 due to <2 trials
Fetched 215 dates from Sessions table from 2024-07-20 to 2025-03-15 for R041

animal_id R043
dropping 0 sessions of 213 due to <2 trials
Fetched 213 dates from Sessions table from 2024-07-20 to 2025-03-15 for R043

animal_id R045
dropping 4 sessions of 207 due to <2 trials
Fetched 203 dates from Sessions table from 2024-07-20 to 2025-03-15 for R045

animal_id R047
dropping 6 sessions of 212 due to <2 trials
Fetched 206 dates from Sessions table from 2024-07-20 to 2025-03-15 for R047

animal_id R049
dropping 1 sessions of 200 due to <2 trials
Fetched 199 dates from Sessions table from 2024-07-29 to 2025-03-15 for R049

animal_id R051
dropping 4 sessions of 210 due to <2 trials
Fetched 206 dates from Sessions table from 2024-07-27 to 2025-03-15 for R051

animal_id R053
dropping 2 sessions of 52 due to <2 trials
Fetched 50 dates from Sessions table from 2024-07-27 to 2024-09-21 for R053

animal_id R055
dropping 9 sessions of 200 d

In [15]:
c_tdf = c_tdf.query("stage == 11 and animal_id != 'R053'") # R053 did not continue training
c_tdf = u.compute_days_relative_to_stage(c_tdf, stage=11)

In [16]:
c_tdf.to_parquet("data/V2_tdf_continued_stage_11.parquet")

# 5. Sb (DMS2) Task Data


In [30]:

DMS_RATS = [
    'R010', 'R011', 'R012', 'R013', 'R014', 'R015', 'R020', 'R021',  'R022', 'R023', 'R024', 'R025', 'R026', 'R027', 'R028', 'R029', 'R030', 'R031', 'R032', 'R033', 'R034', 'R035', 'R036', 'R037', 'R039']

date_min = '2023-04-26'
date_max = '2024_04_26'

In [31]:
trials_df = ingest.create_trials_df.create_trials_df_from_dj(
    animal_ids=DMS_RATS, date_min=date_min, date_max=date_max
)

animal_id R010
Not all variables are the same length. Cannot create proper DataFrame
result: 9
hits: 9
violations: 9
temperror: 9
hit_rate: 8
error_rate: 8
violation_rate: 8
temp_error_rate: 8
n_lpokes: 8
n_cpokes: 8
n_rpokes: 8
trial_dur: 8
trial_start: 8
valid_early_spoke: 8
first_spoke: 8
first_lpoke: 8
first_rpoke: 8
n_settling_ins: 8
avg_settling_in: 8
cpoke_dur: 8
stage: 8
curriculum: 8
sides: 9
SMA_set: 8
is_match: 9
sa: 8
sb: 8
stimuli_on: 8
settling_in_dur: 8
legal_cbreak_dur: 8
pre_dur: 8
adj_pre_dur: 8
stimulus_dur: 8
delay_dur: 8
post_dur: 8
sb_extra_dur: 8
viol_off_dur: 8
pre_go_dur: 8
inter_trial_dur: 8
go_type: 8
go_dur: 8
give_type_set: 8
give_type_imp: 8
give_frac: 8
give_delay_dur: 8
give_xtra_light_delay_dur: 8
give_use: 9
replay_on: 8
timeouts: 9
water_delivered: 9
give_water_not_drunk: 9
crash_hist: 8
was_block_swtich: 8
volume_multiplier: 8
l_water_vol: 8
r_water_vol: 8
ab_l_prob: 8
ab_r_prob: 8
ab_beta: 8
ab_tau: 8
exp_del_tau: 8
exp_del_min: 8
exp_del_max: 8
ses

In [32]:
trials_df.head()

,trial,animal_id,date,sess_id,protocol,result,hits,violations,temperror,hit_rate,...,block_switch_sound_dur,give_del_adagrow_trial_subset,give_del_adagrow_threshold,give_del_adagrow_subset_prev_perf,give_del_adagrow_step_size,give_del_adagrpw_window_size,give_del_adagrow_window_size,give_del_growth_type,give_del_adagrow_alpha_plus,give_del_adagrow_alpha_minus
0,1,R010,2023-08-13,924666,DMS2,6,<NA>,<NA>,<NA>,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,R010,2023-08-13,924666,DMS2,6,<NA>,<NA>,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,R010,2023-08-13,924666,DMS2,6,<NA>,<NA>,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,R010,2023-08-13,924666,DMS2,6,<NA>,<NA>,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,R010,2023-08-13,924666,DMS2,6,<NA>,<NA>,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
columns_of_interest = [
    "trial",
    "animal_id",
    "date",
    "sess_id",
    "hits",
    "violations",
    "stage",
    "sa",
    "sb",
    "stimuli_on",
    "settling_in_dur",
    "legal_cbreak_dur",
    "pre_dur",
    "adj_pre_dur",
    "stimulus_dur",
    "delay_dur",
    "post_dur",
    "sb_extra_dur",
    "viol_off_dur",
    "pre_go_dur",
    "sound_pair",
    "give_type_imp",
    "give_frac",
    "n_settling_ins",
    "avg_settling_in",
    "cpoke_dur",
]

ftdf = trials_df.query("stage >=5 and stage <= 11")[columns_of_interest].copy()

ftdf.to_parquet("data/dms_sb_tdf.parquet", index=False)

# 6. Mouse Fixation Data


In [11]:
ANIMALS = c.MICE
m_tdf = ingest.create_trials_df.create_trials_df_from_dj(
    ANIMALS, date_min="2023-07-12", date_max="2023-12-31"
)


animal_id R512
Fetched 108 dates from Sessions table from 2023-07-12 to 2023-11-09 for R512

animal_id R513
Fetched 106 dates from Sessions table from 2023-07-12 to 2023-11-09 for R513

animal_id R621
Fetched 96 dates from Sessions table from 2023-07-26 to 2023-11-09 for R621

animal_id R622
Fetched 97 dates from Sessions table from 2023-07-26 to 2023-11-09 for R622

animal_id R623
Fetched 99 dates from Sessions table from 2023-07-26 to 2023-11-09 for R623



In [12]:
# only look at cpoke growth stages to sounds on (no sound rule req.)
m_tdf = m_tdf.query("stage >= 5 and stage < 10").copy() 

# add handy columns
m_tdf["dt_date"] = pd.to_datetime(m_tdf['date'])
m_tdf["cumulative_trial"] = m_tdf.groupby("animal_id").cumcount() + 1
m_tdf = u.compute_days_relative_to_stage(m_tdf, stage=5)
m_tdf["session"] = m_tdf.groupby("animal_id")["dt_date"].transform(
    lambda x: (x - x.min()).dt.days + 1
)

# determine fixation duration given the trial structure (specific to data from DMS2 protocol)
m_tdf["fixation_dur"] = m_tdf.apply(u.determine_fixation_dur, axis=1)

m_tdf.to_parquet("data/MouseFixation_tdf.parquet", index=False)